<a href="https://colab.research.google.com/github/ykim71/thesis_related/blob/main/ner_pos_matching_wpreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/'MyDrive'/CrowdTangle/final_data_metrics

/content/drive/MyDrive/CrowdTangle/final_data_metrics




## pre-processing

In [ ]:
# import words/phrases from NER and POS tagging
import pandas as pd

ner_pos = pd.read_excel('pos_bigrams_all_merged_NER4.xlsx')


In [ ]:
import re

def bigrams(s): # more than bigrams
    return [i for i in s if re.search(r'\s', i) ] 
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]
def hashtag(s):
    return [i for i in s if re.search(r'#', i) ]

def contain_the(s): # bigrams that make sense with 'the'
  to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ]
  p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars
  s = [p.sub('', s) for s in s] # remove hashtags

  matchers = ['The ', 'the ']
  items = [i for i in s if any(xs in i for xs in matchers)]
  temp = [i for i in items if i.count(' ')==1]
  return list(set(temp))


In [ ]:
# for Republican words/phrases

ner_rep_only = ner_pos.loc[(ner_pos['rep'] == 1)]

temp = ner_rep_only['text'].values.tolist()
ner_rep_only_list = list(set(temp))
print(len(ner_rep_only_list))

# bi-grams
rep_bi = list(set(bigrams(ner_rep_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_bi = [p.sub('', s) for s in rep_bi] # remove hashtags

rep_bi = list(set(bigrams(rep_bi))) # select only bi-grams (hashtags will be added here)

# uni-grams

rep_uni = list(set(unigrams(ner_rep_only_list)))
print(len(rep_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_uni = [p.sub('', s) for s in rep_uni] # remove some characters
rep_uni = list(set(rep_uni))
print(len(rep_uni))


# hashtags

import re

rep_hash = list(set(hashtag(ner_rep_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_hash = [p.sub('', s) for s in rep_hash] # remove hashtags
rep_hash_non = list(set(rep_hash))

rep_hash = ['#' + s for s in rep_hash]
rep_hash = list(set(rep_hash))

# bigrams with the

rep_bi_the = contain_the(list(set(bigrams(ner_rep_only_list)))) 
print(len(rep_bi_the))

2712
185
181
1


In [ ]:
rep_bi_hashtags = rep_bi + rep_hash + rep_bi_the
rep_uni_nonhashtags = rep_uni + rep_hash_non


In [ ]:
# for Democratic words/phrases

ner_dem_only = ner_pos.loc[(ner_pos['dem'] == 1)]

temp = ner_dem_only['text'].values.tolist()
ner_dem_only_list = list(set(temp))
print(len(ner_dem_only_list))

# bi-grams
dem_bi = list(set(bigrams(ner_dem_only_list)))

import re

to_remove = [' #', '# ', '.','\'s', "’s", " \'", "\'", "\'S", '“', '"', '’', ' Timeline', ' Photo', 'the ', 'The ']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_bi = [p.sub('', s) for s in dem_bi] # remove hashtags

dem_bi = list(set(bigrams(dem_bi))) # select only bi-grams (hashtags will be added here)


# uni-grams

dem_uni = list(set(unigrams(ner_dem_only_list)))
print(len(dem_uni))
import re

to_remove = ['#', '.', '\'s', "’s", " \'", "\'", "\'S", '“', '"', '’']

p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_uni = [p.sub('', s) for s in dem_uni] # remove some characters
dem_uni = list(set(dem_uni))
print(len(dem_uni))

# hashtags

import re

dem_hash = list(set(hashtag(ner_dem_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
dem_hash_non = list(set(dem_hash))

dem_hash = ['#' + s for s in dem_hash]
dem_hash = list(set(dem_hash))


# bigrams with the

dem_bi_the = contain_the(list(set(bigrams(ner_dem_only_list)))) 


2533
175
173


In [ ]:

dem_bi_hashtags = dem_bi + dem_hash + dem_bi_the
dem_uni_nonhashtags = dem_uni + dem_hash_non


In [ ]:
# make lowercases for matching

temp_rep_lower_bi = list( set([x.lower() for x in rep_bi_hashtags] ))
temp_dem_lower_bi = list( set([x.lower() for x in dem_bi_hashtags] ))

temp_rep_lower_uni = list( set([x.lower() for x in rep_uni_nonhashtags]) )
temp_dem_lower_uni = list( set([x.lower() for x in dem_uni_nonhashtags]) )

# temp_rep_lower_bi_the = list( set([x.lower() for x in rep_bi_the] ))
# temp_dem_lower_bi_the = list( set([x.lower() for x in dem_bi_the] ))

In [ ]:
print(len(temp_rep_lower_bi))
print(len(temp_dem_lower_bi))

print(len(temp_rep_lower_uni))
print(len(temp_dem_lower_uni))


2133
2059
245
236


## matching sample

In [54]:
# for entire posts
import re

def basic_sanitize(text):
    
    text = re.sub('’s',' s', text) #&
    text = re.sub('\'s',' s', text) #&
    text = re.sub('’S',' S', text) #&
    text = re.sub('\'S',' S', text) #&

    text = re.sub(r'[^\w\s#&-]','', text) #&
#    text = text.lower()
    
    return text

In [4]:
import pickle
import pandas as pd
ner = pd.read_excel("pos_bigrams_all_merged_NER5_final.xlsx")
pos = pd.read_excel("pos_both_all_no_san_c300_merge_trim.xlsx")


In [55]:
ner_y = ner[ner['NER']=='Y']

In [56]:
ner_y['text_san'] = ner_y['text'].apply(basic_sanitize)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
pos['text_san'] = pos['text'].apply(basic_sanitize)

In [58]:
ner_y[ (ner_y['dem']==1) | (ner_y['rep']==1)].sample(10)

,id,pos_junky,text,count_dem,count_sum,politician,dem,rep,media,Unnamed: 9,NER,text_san
69872,69913,3,Michele Obama,NaN,NaN,0,1,0,0,done,Y,Michele Obama
64365,64399,3,BIDEN,NaN,NaN,1,1,0,0,done,Y,BIDEN
9322,9325,3,Chanel Rion,104.0,515.0,0,0,1,1,done,Y,Chanel Rion
15307,15313,3,Dick Durbin,1481.0,3449.0,1,1,0,0,done,Y,Dick Durbin
29151,29166,3,Jim Moran,116.0,264.0,1,1,0,0,done,Y,Jim Moran
8797,8800,3,Carol Donovan,555.0,555.0,1,1,0,0,done,Y,Carol Donovan
71539,71583,3,Redstate News,NaN,NaN,0,0,1,1,done,Y,Redstate News
29299,29314,3,Joe Wilson,369.0,631.0,1,0,1,0,done,Y,Joe Wilson
43386,43403,3,Peter Beinart,117.0,151.0,0,1,0,1,done,Y,Peter Beinart
34305,34320,3,Martin O’Malley,673.0,955.0,1,1,0,0,done,Y,Martin OMalley


In [65]:
pos[ (pos['dem']==1) | (pos['rep']==1)].sample(2)

,Unnamed: 0,Unnamed: 0.1,text,text_san,check,rep,dem,count_sum,type,count_sum_clean
37820,35440,3600911,John Hoeven,John Hoeven,1,1,0,300,bigram,300.0
29980,28635,2943149,Gov. Charlie,Gov Charlie,1,1,0,774,bigram,785.0


In [66]:
import re

def bigrams(s): # more than bigrams
    return [i for i in s if re.search(r'\s', i) ] 
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]
def hashtag(s):
    return [i for i in s if re.search(r'#', i) ]


In [67]:
# ner

ner_dem_only = ner_y.loc[(ner_y['dem'] >= 1)]

temp = ner_dem_only['text_san'].values.tolist()
ner_dem_only_list = list(set(temp))
print(len(ner_dem_only_list))

dem_bi_ner = list(set(bigrams(ner_dem_only_list)))
print(len(dem_bi_ner))

dem_uni_ner = list(set(unigrams(ner_dem_only_list)))
print(len(dem_uni_ner))

## pos
pos_dem_only = pos.loc[(pos['dem'] >= 1)]

temp = pos_dem_only['text_san'].values.tolist()
pos_dem_only_list = list(set(temp))
print(len(pos_dem_only_list))

dem_bi_pos = list(set(bigrams(pos_dem_only_list)))
print(len(dem_bi_pos))

dem_uni_pos = list(set(unigrams(pos_dem_only_list)))
print(len(dem_uni_pos))


# hash - ner
import re

dem_hash = list(set(hashtag(ner_dem_only_list + pos_dem_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

dem_hash = [p.sub('', s) for s in dem_hash] # remove hashtags
dem_hash_non = list(set(dem_hash))

dem_hash = ['#' + s for s in dem_hash]
dem_hash = list(set(dem_hash))

print(len(dem_hash))
print(len(dem_hash_non))


1439
1277
162
2477
2147
330
75
75


In [68]:
# ner

ner_rep_only = ner_y.loc[(ner_y['rep'] >= 1)]

temp = ner_rep_only['text_san'].values.tolist()
ner_rep_only_list = list(set(temp))
print(len(ner_rep_only_list))

rep_bi_ner = list(set(bigrams(ner_rep_only_list)))
print(len(rep_bi_ner))

rep_uni_ner = list(set(unigrams(ner_rep_only_list)))
print(len(rep_uni_ner))

## pos
pos_rep_only = pos.loc[(pos['rep'] >= 1)]

temp = pos_rep_only['text_san'].values.tolist()
pos_rep_only_list = list(set(temp))
print(len(pos_rep_only_list))

rep_bi_pos = list(set(bigrams(pos_rep_only_list)))
print(len(rep_bi_pos))

rep_uni_pos = list(set(unigrams(pos_rep_only_list)))
print(len(rep_uni_pos))


# hash - ner
import re

rep_hash = list(set(hashtag(ner_rep_only_list + pos_rep_only_list)))

to_remove = [' #', '# ', '#']
p = re.compile('|'.join(map(re.escape, to_remove))) # escape to handle metachars

rep_hash = [p.sub('', s) for s in rep_hash] # remove hashtags
rep_hash_non = list(set(rep_hash))

rep_hash = ['#' + s for s in rep_hash]
rep_hash = list(set(rep_hash))

print(len(rep_hash))
print(len(rep_hash_non))


1613
1438
175
2017
1680
337
85
85


In [69]:
dem_bi = list(set(dem_bi_ner + dem_bi_pos + dem_hash))
print(len(dem_bi))

dem_uni = list(set(dem_uni_ner + dem_uni_pos + dem_hash_non))
print(len(dem_uni))

rep_bi = list(set(rep_bi_ner + rep_bi_pos + rep_hash))
print(len(rep_bi))

rep_uni = list(set(rep_uni_ner + rep_uni_pos + rep_hash_non))
print(len(rep_uni))

2831
372
2392
392


In [70]:

df1 = pd.DataFrame({'dem_bi':dem_bi})
#df1.to_csv('dem_bi.csv')

df2 = pd.DataFrame({'dem_uni':dem_uni})
#df2.to_csv('dem_uni.csv')

df3 = pd.DataFrame({'rep_bi':rep_bi})
#df3.to_csv('rep_bi.csv')

df4 = pd.DataFrame({'rep_uni':rep_uni})
#df4.to_csv('rep_uni.csv')



In [46]:
import pickle
import pandas as pd

temp = pd.read_csv("r_pages_st_sample_lower.csv")

In [47]:
temp['text_san'] = temp['text'].apply(basic_sanitize)

In [48]:

temp['dem_bi'] = temp['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
temp['dem_uni'] = temp['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))

temp['rep_bi'] = temp['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
temp['rep_uni'] = temp['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [49]:
temp.to_csv('r_pages_st_sample_lower2.csv')

In [19]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# all_pages_sample_1k[all_pages_sample_1k['text_san'].str.contains("#")]


## matching

### for Republican posts

In [20]:
import pickle

with open('r_pages_ner_pos.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [21]:
r_pages_st['text_san'] = r_pages_st['text'].apply(basic_sanitize)


In [22]:
r_pages_st['text_san'].sample(3)

6830247    That didnt take very long Jon Gruden Makes His...
1982195     Terrorism Deaths Quadruple Under Obama - Brei...
3697005    JUST IN Fox News Makes Major Impeachment Move ...
Name: text_san, dtype: object

In [16]:
print(len(dem_bi))
print(len(dem_uni))

print(len(rep_bi))
print(len(rep_uni))

2827
372
2390
392


In [24]:
# matching

r_pages_st['dem_bi'] = r_pages_st['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
r_pages_st['dem_uni'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))


In [25]:

r_pages_st['rep_bi'] = r_pages_st['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
r_pages_st['rep_uni'] = r_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [26]:
# saving results

r_pages_st6 = r_pages_st[['quarter','insult','sentistr','dem_bi','dem_uni','rep_bi','rep_uni']]

import pickle

with open('r_pages_ner_pos_final.pickle', 'wb') as handle:
    pickle.dump(r_pages_st6, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [27]:
# manual checking

r_pages_st_sample = r_pages_st.sample(1000)
r_pages_st_sample.to_csv('r_pages_st_sample_lower.csv', encoding='utf-8-sig')


### for Democratic posts

In [17]:
import pickle

with open('d_pages_ner_pos.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [18]:
d_pages_st['text_san'] = d_pages_st['text'].apply(basic_sanitize)


KeyboardInterrupt: ignored

In [ ]:
d_pages_st['text_san'].sample(3)

In [ ]:

d_pages_st['dem_bi'] = d_pages_st['text_san'].apply(lambda x: [i for i in dem_bi if i in x])
d_pages_st['dem_uni'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), dem_uni))


In [ ]:
d_pages_st['rep_bi'] = d_pages_st['text_san'].apply(lambda x: [i for i in rep_bi if i in x])
d_pages_st['rep_uni'] = d_pages_st['text_san'].apply(lambda x: set.intersection(set(x.split(' ')), rep_uni))


In [ ]:
# saving results

d_pages_st6 = d_pages_st[['quarter','insult','sentistr','dem_bi','dem_uni','rep_bi','rep_uni']]

import pickle

with open('d_pages_ner_pos_final.pickle', 'wb') as handle:
    pickle.dump(d_pages_st6, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# manual checking

d_pages_st_sample = d_pages_st.sample(1000)
d_pages_st_sample.to_csv('d_pages_st_sample_lower.csv', encoding='utf-8-sig')


## merge results from both leanings

In [ ]:
import pickle

with open('r_pages_ner_pos_3rd_lowercase.pickle', 'rb') as handle:
    r_pages_st = pickle.load(handle)

In [ ]:

with open('d_pages_ner_pos_3rd_lowercase.pickle', 'rb') as handle:
    d_pages_st = pickle.load(handle)

In [ ]:
r_pages_st['leanings'] = "republican"
d_pages_st['leanings'] = "democratic"

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

d_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(d_pages_st[['sentistr', 'insult']])

r_pages_st[['sentistr_std', 'insult_std']] = scaler.fit_transform(r_pages_st[['sentistr', 'insult']])


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:993: RuntimeWarning: invalid value encountered in subtract
  X -= self.mean_
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:991: RuntimeWarning: invalid value encountered in subtract
  temp = X - T
/usr/local/lib/python3.7/dist-package

In [ ]:

import numpy as np

d_pages_st['ner_pos_dem_b_count'] = d_pages_st['ner_pos_dem_b'].str.len()
d_pages_st['ner_pos_dem_u_count'] = d_pages_st['ner_pos_dem_u'].str.len()

d_pages_st['ner_pos_rep_b_count'] = d_pages_st['ner_pos_rep_b'].str.len()
d_pages_st['ner_pos_rep_u_count'] = d_pages_st['ner_pos_rep_u'].str.len()


In [ ]:

d_pages_st['out_group_np'] = np.where(( (d_pages_st.ner_pos_rep_u_count >= 1) | (d_pages_st.ner_pos_rep_b_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['in_group_np'] = np.where(( (d_pages_st.ner_pos_dem_u_count >= 1) | (d_pages_st.ner_pos_dem_b_count >= 1) ), 'mentioned', 'non_mentioned')
d_pages_st['no_entity_np'] = np.where(((d_pages_st.in_group_np == 'non_mentioned') & (d_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')
d_pages_st['both_entity'] = np.where(((d_pages_st.in_group_np == 'non_mentioned') & (d_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')


In [ ]:
import numpy as np

r_pages_st['ner_pos_dem_b_count'] = r_pages_st['ner_pos_dem_b'].str.len()
r_pages_st['ner_pos_dem_u_count'] = r_pages_st['ner_pos_dem_u'].str.len()

r_pages_st['ner_pos_rep_b_count'] = r_pages_st['ner_pos_rep_b'].str.len()
r_pages_st['ner_pos_rep_u_count'] = r_pages_st['ner_pos_rep_u'].str.len()


r_pages_st['in_group_np'] = np.where(( (r_pages_st.ner_pos_rep_u_count >= 1) | (r_pages_st.ner_pos_rep_b_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['out_group_np'] = np.where(( (r_pages_st.ner_pos_dem_u_count >= 1) | (r_pages_st.ner_pos_dem_b_count >= 1) ), 'mentioned', 'non_mentioned')
r_pages_st['no_entity_np'] = np.where(((r_pages_st.in_group_np == 'non_mentioned') & (r_pages_st.out_group_np == 'non_mentioned')), 'none', 'mentioned')


In [ ]:

print('in_group_np')
print(d_pages_st['in_group_np'].value_counts())

print('out_group_np')
print(d_pages_st['out_group_np'].value_counts())

print('no_entity_np')
print(d_pages_st['no_entity_np'].value_counts())

in_group_np
non_mentioned    4356840
mentioned        2453184
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    3651227
mentioned        3158797
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4302969
none         2507055
Name: no_entity_np, dtype: int64


In [ ]:

print('in_group_np')
print(r_pages_st['in_group_np'].value_counts())

print('out_group_np')
print(r_pages_st['out_group_np'].value_counts())

print('no_entity_np')
print(r_pages_st['no_entity_np'].value_counts())

in_group_np
non_mentioned    4362780
mentioned        2987858
Name: in_group_np, dtype: int64
out_group_np
non_mentioned    4674140
mentioned        2676498
Name: out_group_np, dtype: int64
no_entity_np
mentioned    4255938
none         3094700
Name: no_entity_np, dtype: int64


In [ ]:
d_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'ner_pos_rep_u', 'ner_pos_rep_b',
       'ner_pos_dem_u', 'ner_pos_dem_b', 'leanings', 'sentistr_std',
       'insult_std', 'ner_pos_dem_b_count', 'ner_pos_dem_u_count',
       'ner_pos_rep_b_count', 'ner_pos_rep_u_count', 'out_group_np',
       'in_group_np', 'no_entity_np'],
      dtype='object')

In [ ]:
r_pages_st.columns

Index(['quarter', 'insult', 'sentistr', 'ner_pos_rep_u', 'ner_pos_rep_b',
       'ner_pos_dem_u', 'ner_pos_dem_b', 'leanings', 'sentistr_std',
       'insult_std', 'ner_pos_dem_b_count', 'ner_pos_dem_u_count',
       'ner_pos_rep_b_count', 'ner_pos_rep_u_count', 'in_group_np',
       'out_group_np', 'no_entity_np'],
      dtype='object')

In [ ]:
import pandas as pd

pages_df_st = pd.concat([r_pages_st, d_pages_st], ignore_index=True, sort=False)


In [ ]:
# aggregated by all posts

temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_no_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/all_np_no_entities_insult_std.csv', encoding='utf-8-sig')


In [ ]:
# aggregated by all posts + by leanings

temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_out_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_in_group_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['sentistr_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_no_entities_senti_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'out_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_out_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'in_group_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_in_group_insult_std.csv', encoding='utf-8-sig')


temp  = pages_df_st.groupby(['quarter', 'no_entity_np', 'leanings'])['insult_std'].mean().reset_index()
temp.to_csv('pos_ner_lower/leanings_np_no_entities_insult_std.csv', encoding='utf-8-sig')
